In [116]:
from sklearn.decomposition import PCA, TruncatedSVD
from scipy import sparse
import pickle
import pandas as pd
import numpy as np

from Constants import *
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE
from sklearn.metrics import pairwise_distances

### Load data

In [117]:
newspaper_speaker_tfidf = sparse.load_npz(FILE_NEWSPAPER_SPEAKER_TFIDF)
newspaper_speaker_tfidf = newspaper_speaker_tfidf.toarray()

newspaper_token_tfidf = sparse.load_npz(FILE_NEWSPAPER_TOKEN_TFIDF)
newspaper_token_tfidf = newspaper_token_tfidf.toarray()

In [118]:
with open(PICKLE_NEWSPAPER_TO_INDEX, 'rb') as handle:
    newspaper_to_index = pickle.load(handle)

with open(PICKLE_INDEX_TO_NEWSPAPER, 'rb') as handle:
    index_to_newspaper = pickle.load(handle)
    
    
with open(PICKLE_SPEAKER_TO_INDEX, 'rb') as handle:
    speaker_to_index = pickle.load(handle)

with open(PICKLE_INDEX_TO_SPEAKER, 'rb') as handle:
    index_to_speaker = pickle.load(handle)
    
with open(PICKLE_INDEX_TO_TOKEN, 'rb') as handle:
    index_to_token = pickle.load(handle)

with open(PICKLE_TOKEN_TO_INDEX, 'rb') as handle:
    token_to_index = pickle.load(handle)
    
newspaper_owner = pd.read_csv(FILE_NEWSPAPER_OWNER)

### Select newspaper that are owned by a group with multiple newspaper

In [119]:
#Minimum number of newspaper needed in a group to consider that group
MIN_NEWSPAPER_COUNT = 4

newspaper_with_owner = newspaper_owner[~newspaper_owner["owner"].isnull()]

print(f'We sucessfully retrieved: {len(newspaper_with_owner) / len(newspaper_owner) * 100}% of the data')

num_np_by_owner = newspaper_with_owner.groupby("owner").count()["newspaper"]

group_owner = set(num_np_by_owner[num_np_by_owner >= MIN_NEWSPAPER_COUNT].index)

newspaper_in_group = newspaper_with_owner[newspaper_with_owner["owner"].isin(group_owner)]

newspaper_in_group

We sucessfully retrieved: 13.61043194784026% of the data


,newspaper,QID,website,owner
2,1045theteam,Q7956286,http://www.1045theteam.com,Townsquare Media
4,107jamz,Q6331512,http://www.107jamz.com,Townsquare Media
5,10news,Q3191396,http://www.10news.com/,E. W. Scripps Company
8,13abc,Q2386816,http://www.13abc.com/,Gray Television
16,247sports,Q16973497,https://247sports.com/,ViacomCBS Streaming
...,...,...,...,...
7269,www,Q150248,https://www.metacritic.com/,ViacomCBS Streaming
7285,wxyz,Q3564870,http://www.wxyz.com/,E. W. Scripps Company
7293,wyrk,Q7958139,http://www.wyrk.com,Townsquare Media
7310,yardbarker,Q8049272,http://www.yardbarker.com/,Fox Entertainment Group


In [120]:
newspaper_in_group['owner'] = pd.Categorical(newspaper_in_group.owner)
newspaper_in_group['owner_ID'] = newspaper_in_group['owner'].cat.codes

/var/folders/c6/nkvhnnln1739v3k9ryxjwq700000gn/T/ipykernel_97834/826135210.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newspaper_in_group['owner'] = pd.Categorical(newspaper_in_group.owner)
/var/folders/c6/nkvhnnln1739v3k9ryxjwq700000gn/T/ipykernel_97834/826135210.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newspaper_in_group['owner_ID'] = newspaper_in_group['owner'].cat.codes


In [121]:
newspaper_in_group

,newspaper,QID,website,owner,owner_ID
2,1045theteam,Q7956286,http://www.1045theteam.com,Townsquare Media,38
4,107jamz,Q6331512,http://www.107jamz.com,Townsquare Media,38
5,10news,Q3191396,http://www.10news.com/,E. W. Scripps Company,10
8,13abc,Q2386816,http://www.13abc.com/,Gray Television,18
16,247sports,Q16973497,https://247sports.com/,ViacomCBS Streaming,41
...,...,...,...,...,...
7269,www,Q150248,https://www.metacritic.com/,ViacomCBS Streaming,41
7285,wxyz,Q3564870,http://www.wxyz.com/,E. W. Scripps Company,10
7293,wyrk,Q7958139,http://www.wyrk.com,Townsquare Media,38
7310,yardbarker,Q8049272,http://www.yardbarker.com/,Fox Entertainment Group,12


In [122]:
print(f'We obtain {newspaper_in_group["owner"].nunique()} different groups')

We obtain 46 different groups


### Project speaker TFIDF in 2D using PCA /TSNE an create a map

In [123]:
def project_PCA(selected_newspapers, newspaper_to_index, X, dim=2):
    newspaper_in_group_idx = [newspaper_to_index[newspaper] for newspaper in selected_newspapers]
    selected_X = X[newspaper_in_group_idx]
    
    pca = PCA(n_components=dim)
    Y = pca.fit_transform(selected_X)
    return Y

In [124]:
def project2D_TSNE(selected_newspapers, newspaper_to_index, X, dim=2):
    newspaper_in_group_idx = [newspaper_to_index[newspaper] for newspaper in selected_newspapers]
    selected_X = X[newspaper_in_group_idx]
    
    distance_matrix = pairwise_distances(selected_X, metric='cosine')
    embeddings = TSNE(n_components=dim,metric='precomputed',square_distances=True)
    Y_tsne = embeddings.fit_transform(distance_matrix)
    
    return Y_tsne

### Bokeh plot of the above map

In [125]:
import bokeh
import bokeh.plotting as bpl
from bokeh.plotting import figure, output_file, show, save
from bokeh.palettes import Turbo256, Plasma, cividis, magma
from bokeh.models import LabelSet,Scatter,HoverTool, WheelZoomTool, PanTool, BoxZoomTool, ResetTool, TapTool, SaveTool, OpenURL
from bokeh.models.callbacks import CustomJS
from bokeh.transform import factor_cmap
bokeh.__version__

'2.4.2'

In [ ]:
def get_mean_distance_to_center(Y, newspaper_in_group):
    df = pd.DataFrame(
        {
            "x": Y[:, 0],
            "y": Y[:, 1],
            "group": newspaper_in_group.owner
        }
    )
    
    mean_per_group = df.groupby("group").mean()
    df = df.join(mean_per_group,on ="group", rsuffix="_mean")
    df["diff"] = np.abs(df["x"] - df["x_mean"]) + np.abs(df["y"] - df["y_mean"])
    return df.groupby("group")["diff"].mean().sort_values()

In [126]:
def plot_scatter_bokeh(Y, newspaper_in_group, visible_by_default = False):
    df = pd.DataFrame(
        {
            "x": Y[:, 0],
            "y": Y[:, 1],
            "group": newspaper_in_group.owner,
            "newspaper": newspaper_in_group.newspaper,
            "QID": newspaper_in_group.QID,
            "website": newspaper_in_group.website
        }
    )
    
    top_culster_group = set(get_mean_distance_to_center(Y, newspaper_in_group).head(10).index)

    # Show info on hover
    hover = HoverTool(tooltips=[('Nespaper', '@newspaper'),('Group name', '@group'),('QID', '@QID'),('Website', '@website')])

    tools = [hover, WheelZoomTool(), PanTool(), BoxZoomTool(), ResetTool(), SaveTool(), TapTool()]

    groups = df['group'].unique()

    color_map = factor_cmap("group", factors=sorted(groups),palette=magma(len(groups)))

    # create figure and plot
    p = bpl.figure(height=1000, width=1500, tools=tools)

    for group_name,points in df.groupby('group'):

        source = bpl.ColumnDataSource(points)

        # Display newspaper
        scat = p.scatter(x='x', y='y',
                  fill_color= color_map,
                  line_color = color_map,
                  legend_label=group_name, source=source)

        

        if not visible_by_default:
            scat.visible = group_name in top_culster_group
        else:
            scat.visible = True
        

        # Open website on click
        source.selected.js_on_change('indices', CustomJS(args=dict(s1=source), code="""
            const inds = cb_obj.indices;
            for (var i = 0; i < inds.length; i++) {
                var ind = inds[i]
                var url = s1.data['website'][ind]
                window.open(url)
        }
            """)
        )
            
    # Show newspaper of the group when clic on the legend
    p.legend.click_policy="hide"

    # Define legend size
    p.legend.label_text_font_size = '16px'
    #Padding on the overall rectangle
    p.legend.padding = 0
    p.legend.spacing = 0
    #Size of circular points
    p.legend.glyph_height = 16
    p.legend.glyph_width = 16
    #Padding between label entries
    p.legend.label_height = 0
    p.legend.label_width = 0
    
    # Generate the graph
    bpl.show(p)

In [127]:
Y =  project_PCA(newspaper_in_group['newspaper'], newspaper_to_index, newspaper_speaker_tfidf)
p = plot_scatter_bokeh(Y,newspaper_in_group)
output_file('newspaper_speaker_PCA.html',mode='inline')
try: 
    save(p)
except:
    pass

In [128]:
Y =  project_PCA(newspaper_in_group['newspaper'], newspaper_to_index, newspaper_token_tfidf)
p = plot_scatter_bokeh(Y,newspaper_in_group)
output_file('newspaper_token_PCA.html',mode='inline')
try: 
    save(p)
except:
    pass

### SVD

In [129]:
import scipy
from sklearn.utils.extmath import randomized_svd
from scipy.sparse.linalg import svds

In [133]:
def project_SVD(selected_newspapers, newspaper_to_index, X, dim=2):
    
    newspaper_in_group_idx = [newspaper_to_index[newspaper] for newspaper in selected_newspapers]
    selected_X = X[newspaper_in_group_idx]
    
    #Same as TruncatedSVD
    U, sigmas, VT = randomized_svd(selected_X, n_components=dim)
    
    X_red = U@np.diag(sigmas)
    
    return X_red, VT

def get_top_axis(VT,index_map,top_num=5):
    
    for i in range(VT.shape[0]):
        max_indices = np.argsort(-VT[i])[:top_num]
        print(f'\nThe maximum values for axis {i} are:\n')
        for index in max_indices:
            print(index_map[index])
        
        
        min_indices = np.argsort(VT[i])[:top_num]
        print(f'\nThe minimum for axis {i} are:\n')
        for index in min_indices:
            print(index_map[index])

#### Newspaper speaker

In [146]:
X_red,VT = project_SVD(newspaper_in_group['newspaper'], newspaper_to_index, newspaper_speaker_tfidf,dim=5)
get_top_axis(VT,index_to_speaker,top_num=10)
#Choose wanted axis to plot 
p = plot_scatter_bokeh(X_red[:,1:3],newspaper_in_group)


The maximum values for axis 0 are:

president donald trump
boris johnson
matt hancock
jose mourinho
nicola sturgeon
pep guardiola
rishi sunak
tedros adhanom ghebreyesus
leo varadkar
frank lampard

The minimum for axis 0 are:

 palmer
 rowdy  roddy piper
 superstar  billy graham
 yoshida
justin gamble
marc horowitz
marc huber
marc humbert
marc j cohen
marc janssen

The maximum values for axis 1 are:

president donald trump
joe biden
bernie sanders
andrew cuomo
anthony fauci
elizabeth warren
nancy pelosi
gavin newsom
pete buttigieg
mitch mcconnell

The minimum for axis 1 are:

boris johnson
matt hancock
jose mourinho
pep guardiola
nicola sturgeon
rishi sunak
leo varadkar
frank lampard
mary lou mcdonald
jurgen klopp

The maximum values for axis 2 are:

scott morrison
brendan murphy
annastacia palaszczuk
steven miles
greg hunt
karl stefanovic
gladys berejiklian
paul kelly
brad hazzard
josh frydenberg

The minimum for axis 2 are:

president donald trump
joe biden
bernie sanders
gavin newso

##### Speaker clustering analysis

Axis 0:
- Max: Popular speakers
- Min: Unpopular speakers

Axis 1:
- Max: US speakers
- Min: UK speakers

Axis 2:
- Max: Australian speakers
- Min: US speakers

Axis 3:
- Max: Canadian speakers
- Min: US politicians speakers

Axis 4:
- Max: Basketball speakers
- Min: US politicians speakers

#### Newspaper token

In [147]:
X_red,VT = project_SVD(newspaper_in_group['newspaper'], newspaper_to_index, newspaper_token_tfidf,dim=5)
get_top_axis(VT,index_to_token,top_num=10)
#Choose wanted axis to plot 
p = plot_scatter_bokeh(X_red[:,1:3],newspaper_in_group)


The maximum values for axis 0 are:

people
going
get
time
like
think
one
would
know
really

The minimum for axis 0 are:

AAAAA
AAAAAA
AAAAAAA
AAAAHHHH
AAAP
AAB
NurtureCloud
golem
goldspot
Nurul

The maximum values for axis 1 are:

game
player
play
team
good
got
club
guy
season
really

The minimum for axis 1 are:

people
state
health
community
Trump
president
country
public
need
COVID

The maximum values for axis 2 are:

UK
NHS
club
Government
player
need
support
Scotland
advice
Ireland

The minimum for axis 2 are:

like
guy
going
song
music
know
lot
really
kind
love

The maximum values for axis 3 are:

music
song
like
love
album
life
band
fucking
show
UK

The minimum for axis 3 are:

game
going
team
guy
state
play
player
coach
win
community

The maximum values for axis 4 are:

Canada
Ontario
community
Canadian
Canadians
government
province
puck
year
Indigenous

The minimum for axis 4 are:

president
Trump
American
player
club
President
know
UK
state
going


##### Tokens clustering analysis

Axis 0:
- Max: Popular words
- Min: Unpopular words (rare)

Axis 1:
- Max: Sport
- Min: US health

Axis 2:
- Max: UK sport/politic
- Min: Music/Joy: garbage

Axis 3:
- Max: Music
- Min: Sport

Axis 4:
- Max: Canada
- Min: US/UK : garbage